# 2. Develop your segmentation model

In [8]:
#Import libraries
import os
from model_class import ModelSegmentation
from aux_functions import slicing
import cv2
#Inputs
working_directory="C:/Users/Pheno/Documents/database_almondcv2/"
pictures_directory=os.path.join(working_directory, "strawberry/strawberry_ejemplos")
pre_model=os.path.join(working_directory, "models/yolo11s-seg.pt")
model_path=os.path.join(working_directory, "models/strawberry.pt")
output_directory=os.path.join(working_directory,"output_directory")

### Slicing

In [ ]:
#Slice_pictures for training
slicing(input_folder=pictures_directory,output_directory=working_directory,name_slicing="Slices_probando", number_pictures=4, slice_height=320, slice_width=320)

### Training

In [ ]:
#Label with CVAT

zip_file_shell=os.path.join(working_directory,"strawberry/strawberry_2025.zip")

In [ ]:
#Model segmentation training

model=ModelSegmentation(working_directory=working_directory)
model.train_segmentation_model(input_zip=zip_file_shell, epochs=50,imgsz=640, name_segmentation="strawberry",
                                      pre_model=pre_model, batch=16)

### Deploy and reconstruct a picture

#### Slice_predict_reconstruct approach

In [9]:
# Join patches approach

model=ModelSegmentation(working_directory=working_directory)
masks=model.slice_predict_reconstruct(input_folder=pictures_directory,imgsz=320, model_path=model_path,
                                          slice_height=640, slice_width=640,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)

02/12/2025 12:36:38 - INFO - sahi.slicing -   image.shape: (3008, 1688)
02/12/2025 12:36:38 - INFO - sahi.slicing -   Num slices: 24 slice_height: 640 slice_width: 640


Detected GPU: NVIDIA GeForce RTX 3060
Total GPU Memory: 12.00 GB
Image 1/10


02/12/2025 12:36:43 - INFO - sahi.slicing -   image.shape: (3008, 1688)
02/12/2025 12:36:43 - INFO - sahi.slicing -   Num slices: 24 slice_height: 640 slice_width: 640


Image 2/10


02/12/2025 12:36:47 - INFO - sahi.slicing -   image.shape: (3008, 1688)
02/12/2025 12:36:47 - INFO - sahi.slicing -   Num slices: 24 slice_height: 640 slice_width: 640


...

Image 8/10


02/12/2025 12:37:16 - INFO - sahi.slicing -   image.shape: (3008, 1688)
02/12/2025 12:37:16 - INFO - sahi.slicing -   Num slices: 24 slice_height: 640 slice_width: 640


Image 9/10


02/12/2025 12:37:30 - INFO - sahi.slicing -   image.shape: (3008, 1688)
02/12/2025 12:37:30 - INFO - sahi.slicing -   Num slices: 24 slice_height: 640 slice_width: 640


Image 10/10


In [10]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    cv2.imwrite(f"{output_directory}/{os.path.basename(mask[1])}", mask[0])


#### SAHI

In [24]:
model=ModelSegmentation(working_directory=working_directory)
masks=model.predict_model_sahi(model_path=model_path, check_result=False, folder_input=pictures_directory,
                                            retina_masks=True,
                                              postprocess_match_threshold=0.2, overlap_height_ratio=0.2,
                                                overlap_width_ratio=0.2, postprocess_match_metric="IOS", 
                                                postprocess_type="GREEDYNMM", slice_height=640, slice_width=640,
                                                  confidence_treshold=0.8,
                                                  imgsz=640)

Detected GPU: NVIDIA GeForce RTX 3060
Total GPU Memory: 12.00 GB
Pic 1/10
Performing prediction on 24 slices.
Pic 2/10
Performing prediction on 24 slices.
Pic 3/10
Performing prediction on 24 slices.
Pic 4/10
Performing prediction on 24 slices.
Pic 5/10
Performing prediction on 24 slices.
Pic 6/10
Performing prediction on 24 slices.
Pic 7/10
Performing prediction on 24 slices.
Pic 8/10
Performing prediction on 24 slices.
Pic 9/10
Performing prediction on 24 slices.
Pic 10/10
Performing prediction on 24 slices.


In [23]:
# To show the masks
os.makedirs(output_directory, exist_ok=True)
for mask in masks:
    mask[0].export_visuals(export_dir=output_directory, hide_labels=True, rect_th=1, file_name=f"{os.path.basename(mask[1])}")